<a href="https://colab.research.google.com/github/theclassofai/Brain_Computer_Interfacing/blob/master/main_Analysis_COVID.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%matplotlib inline
import pandas as pd
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
pd.set_option('display.max_columns', 99)
pd.set_option('display.max_rows', 99)
import os
import numpy as np
from matplotlib import pyplot as plt
from tqdm import tqdm
import datetime as dt

In [2]:
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [16, 10]
plt.rcParams['font.size'] = 14
import seaborn as sns
sns.set_palette(sns.color_palette('tab20', 20))

import plotly.express as px
import plotly.graph_objects as go

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
import os
os.environ['KAGGLE_USERNAME'] = "theclassofai" # username from the json file
os.environ['KAGGLE_KEY'] = "2209b8616e7610de0208f66027560847" # key from the json file
!kaggle competitions download -c covid19-global-forecasting-week-2



# !git clone https://github.com/CSSEGISandData/COVID-19.git 

train.csv: Skipping, found more recently modified local copy (use --force to force download)
test.csv: Skipping, found more recently modified local copy (use --force to force download)
submission.csv: Skipping, found more recently modified local copy (use --force to force download)


In [0]:
COMP = '/content'
DATEFORMAT = '%Y-%m-%d'


def get_comp_data(COMP):
    train = pd.read_csv(f'{COMP}/train.csv')
    test = pd.read_csv(f'{COMP}/test.csv')
    submission = pd.read_csv(f'{COMP}/submission.csv')
    print(train.shape, test.shape, submission.shape)
    train['Country_Region'] = train['Country_Region'].str.replace(',', '')
    test['Country_Region'] = test['Country_Region'].str.replace(',', '')

    train['Location'] = train['Country_Region'] + '-' + train['Province_State'].fillna('')

    test['Location'] = test['Country_Region'] + '-' + test['Province_State'].fillna('')

    train['LogConfirmed'] = to_log(train.ConfirmedCases)
    train['LogFatalities'] = to_log(train.Fatalities)
    train = train.drop(columns=['Province_State'])
    test = test.drop(columns=['Province_State'])

    country_codes = pd.read_csv('/content/country_codes.csv', keep_default_na=False)
    train = train.merge(country_codes, on='Country_Region', how='left')
    test = test.merge(country_codes, on='Country_Region', how='left')

    train['DateTime'] = pd.to_datetime(train['Date'])
    test['DateTime'] = pd.to_datetime(test['Date'])
    
    return train, test, submission


def process_each_location(df):
    dfs = []
    for loc, df in tqdm(df.groupby('Location')):
        df = df.sort_values(by='Date')
        df['Fatalities'] = df['Fatalities'].cummax()
        df['ConfirmedCases'] = df['ConfirmedCases'].cummax()
        df['LogFatalities'] = df['LogFatalities'].cummax()
        df['LogConfirmed'] = df['LogConfirmed'].cummax()
        df['LogConfirmedNextDay'] = df['LogConfirmed'].shift(-1)
        df['ConfirmedNextDay'] = df['ConfirmedCases'].shift(-1)
        df['DateNextDay'] = df['Date'].shift(-1)
        df['LogFatalitiesNextDay'] = df['LogFatalities'].shift(-1)
        df['FatalitiesNextDay'] = df['Fatalities'].shift(-1)
        df['LogConfirmedDelta'] = df['LogConfirmedNextDay'] - df['LogConfirmed']
        df['ConfirmedDelta'] = df['ConfirmedNextDay'] - df['ConfirmedCases']
        df['LogFatalitiesDelta'] = df['LogFatalitiesNextDay'] - df['LogFatalities']
        df['FatalitiesDelta'] = df['FatalitiesNextDay'] - df['Fatalities']
        dfs.append(df)
    return pd.concat(dfs)


def add_days(d, k):
    return dt.datetime.strptime(d, DATEFORMAT) + dt.timedelta(days=k)


def to_log(x):
    return np.log(x + 1)


def to_exp(x):
    return np.exp(x) - 1

In [8]:
start = dt.datetime.now()
train, test, submission = get_comp_data(COMP)
train.shape, test.shape, submission.shape
train.head(2)
test.head(2)

(20580, 6) (12642, 4) (12642, 3)


((20580, 13), (12642, 9), (12642, 3))

,Id,Country_Region,Date,ConfirmedCases,Fatalities,Location,LogConfirmed,LogFatalities,country_iso_code_2,country_iso_code_3,continent,geo_region,DateTime
0,1,Afghanistan,2020-01-22,0.0,0.0,Afghanistan-,0.0,0.0,AF,AFG,Asia,Southern Asia,2020-01-22
1,2,Afghanistan,2020-01-23,0.0,0.0,Afghanistan-,0.0,0.0,AF,AFG,Asia,Southern Asia,2020-01-23


,ForecastId,Country_Region,Date,Location,country_iso_code_2,country_iso_code_3,continent,geo_region,DateTime
0,1,Afghanistan,2020-03-19,Afghanistan-,AF,AFG,Asia,Southern Asia,2020-03-19
1,2,Afghanistan,2020-03-20,Afghanistan-,AF,AFG,Asia,Southern Asia,2020-03-20


In [9]:
train.describe()
train.nunique()
train.dtypes
train.count()

TRAIN_START = train.Date.min()
TEST_START = test.Date.min()
TRAIN_END = train.Date.max()
TEST_END = test.Date.max()
TRAIN_START, TRAIN_END, TEST_START, TEST_END

,Id,ConfirmedCases,Fatalities,LogConfirmed,LogFatalities
count,20580.000000,20580.000000,20580.000000,20580.000000,20580.000000
mean,14685.500000,514.939116,21.080952,1.819020,0.384179
std,8487.230117,4541.261768,287.758197,2.526578,1.038621
min,1.000000,0.000000,0.000000,0.000000,0.000000
25%,7335.750000,0.000000,0.000000,0.000000,0.000000
50%,14685.500000,0.000000,0.000000,0.000000,0.000000
75%,22035.250000,35.000000,0.000000,3.583519,0.000000
max,29370.000000,105792.000000,12428.000000,11.569240,9.427788


Id                    20580
Country_Region          173
Date                     70
ConfirmedCases         1536
Fatalities              328
Location                294
LogConfirmed           1536
LogFatalities           328
country_iso_code_2      173
country_iso_code_3      173
continent                 6
geo_region               21
DateTime                 70
dtype: int64

Id                             int64
Country_Region                object
Date                          object
ConfirmedCases               float64
Fatalities                   float64
Location                      object
LogConfirmed                 float64
LogFatalities                float64
country_iso_code_2            object
country_iso_code_3            object
continent                     object
geo_region                    object
DateTime              datetime64[ns]
dtype: object

Id                    20580
Country_Region        20580
Date                  20580
ConfirmedCases        20580
Fatalities            20580
Location              20580
LogConfirmed          20580
LogFatalities         20580
country_iso_code_2    20580
country_iso_code_3    20580
continent             20580
geo_region            20580
DateTime              20580
dtype: int64

('2020-01-22', '2020-03-31', '2020-03-19', '2020-04-30')

In [10]:
train = train.sort_values(by='Date')
countries_latest_state = train[train['Date'] == TRAIN_END].groupby([
    'Country_Region', 'continent', 'geo_region', 'country_iso_code_3']).sum()[[
    'ConfirmedCases', 'Fatalities']].reset_index()
countries_latest_state['Log10Confirmed'] = np.log10(countries_latest_state.ConfirmedCases + 1)
countries_latest_state['Log10Fatalities'] = np.log10(countries_latest_state.Fatalities + 1)
countries_latest_state = countries_latest_state.sort_values(by='Fatalities', ascending=False)
countries_latest_state.to_csv('countries_latest_state.csv', index=False)

countries_latest_state.shape
countries_latest_state.head()

(173, 8)

,Country_Region,continent,geo_region,country_iso_code_3,ConfirmedCases,Fatalities,Log10Confirmed,Log10Fatalities
81,Italy,Europe,Southern Europe,ITA,105792.0,12428.0,5.024457,4.094436
147,Spain,Europe,Southern Europe,ESP,95923.0,8464.0,4.981927,3.927627
162,US,Americas,Northern America,USA,188018.0,3870.0,5.274202,3.587823
58,France,Europe,Western Europe,FRA,52827.0,3532.0,4.722864,3.548144
33,China,Asia,Eastern Asia,CHN,82279.0,3309.0,4.915294,3.519828


In [11]:
fig = go.Figure(data=go.Choropleth(
    locations = countries_latest_state['country_iso_code_3'],
    z = countries_latest_state['Log10Confirmed'],
    text = countries_latest_state['Country_Region'],
    colorscale = 'viridis_r',
    autocolorscale=False,
    reversescale=False,
    marker_line_color='darkgray',
    marker_line_width=0.5,
    colorbar_tickprefix = '10^',
    colorbar_title = 'Confirmed cases <br>(log10 scale)',
))

_ = fig.update_layout(
    title_text=f'COVID-19 Global Cases [Updated: {TRAIN_END}]',
    geo=dict(
        showframe=False,
        showcoastlines=False,
        projection_type='equirectangular'
    )
)

fig.show()

In [12]:
fig = go.Figure(data=go.Choropleth(
    locations = countries_latest_state['country_iso_code_3'],
    z = countries_latest_state['Log10Fatalities'],
    text = countries_latest_state['Country_Region'],
    colorscale = 'viridis_r',
    autocolorscale=False,
    reversescale=False,
    marker_line_color='darkgray',
    marker_line_width=0.5,
    colorbar_tickprefix = '10^',
    colorbar_title = 'Deaths <br>(log10 scale)',
))

_ = fig.update_layout(
    title_text=f'COVID-19 Global Deaths [Updated: {TRAIN_END}]',
    geo=dict(
        showframe=False,
        showcoastlines=False,
        projection_type='equirectangular'
    )
)

fig.show()

In [13]:
countries_latest_state['DeathConfirmedRatio'] = (countries_latest_state.Fatalities + 1) / (countries_latest_state.ConfirmedCases + 1)
countries_latest_state['DeathConfirmedRatio'] = countries_latest_state['DeathConfirmedRatio'].clip(0, 0.1) 
fig = px.scatter(countries_latest_state,
                 x='ConfirmedCases',
                 y='Fatalities',
                 color='DeathConfirmedRatio',
                 size='Log10Fatalities',
                 size_max=20,
                 hover_name='Country_Region',
                 color_continuous_scale='viridis_r'
)
_ = fig.update_layout(
    title_text=f'COVID-19 Deaths vs Confirmed Cases by Country [Updated: {TRAIN_END}]',
    xaxis_type="log",
    yaxis_type="log"
)
fig.show()

In [14]:
train_clean = process_each_location(train)

train_clean.shape
train_clean.tail()

100%|██████████| 294/294 [00:02<00:00, 130.20it/s]


(20580, 22)

,Id,Country_Region,Date,ConfirmedCases,Fatalities,Location,LogConfirmed,LogFatalities,country_iso_code_2,country_iso_code_3,continent,geo_region,DateTime,LogConfirmedNextDay,ConfirmedNextDay,DateNextDay,LogFatalitiesNextDay,FatalitiesNextDay,LogConfirmedDelta,ConfirmedDelta,LogFatalitiesDelta,FatalitiesDelta
20575,29366,Zimbabwe,2020-03-27,5.0,1.0,Zimbabwe-,1.791759,0.693147,ZW,ZWE,Africa,Eastern Africa,2020-03-27,2.079442,7.0,2020-03-28,0.693147,1.0,0.287682,2.0,0.0,0.0
20576,29367,Zimbabwe,2020-03-28,7.0,1.0,Zimbabwe-,2.079442,0.693147,ZW,ZWE,Africa,Eastern Africa,2020-03-28,2.079442,7.0,2020-03-29,0.693147,1.0,0.000000,0.0,0.0,0.0
20577,29368,Zimbabwe,2020-03-29,7.0,1.0,Zimbabwe-,2.079442,0.693147,ZW,ZWE,Africa,Eastern Africa,2020-03-29,2.079442,7.0,2020-03-30,0.693147,1.0,0.000000,0.0,0.0,0.0
20578,29369,Zimbabwe,2020-03-30,7.0,1.0,Zimbabwe-,2.079442,0.693147,ZW,ZWE,Africa,Eastern Africa,2020-03-30,2.197225,8.0,2020-03-31,0.693147,1.0,0.117783,1.0,0.0,0.0
20579,29370,Zimbabwe,2020-03-31,8.0,1.0,Zimbabwe-,2.197225,0.693147,ZW,ZWE,Africa,Eastern Africa,2020-03-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [0]:
regional_progress = train_clean.groupby(['DateTime', 'continent']).sum()[['ConfirmedCases', 'Fatalities']].reset_index()
regional_progress['Log10Confirmed'] = np.log10(regional_progress.ConfirmedCases + 1)
regional_progress['Log10Fatalities'] = np.log10(regional_progress.Fatalities + 1)
regional_progress = regional_progress[regional_progress.continent != '#N/A']

In [16]:
fig = px.area(regional_progress, x="DateTime", y="ConfirmedCases", color="continent")
_ = fig.update_layout(
    title_text=f'COVID-19 Cumulative Confirmed Cases by Continent [Updated: {TRAIN_END}]'
)
fig.show()
fig2 = px.line(regional_progress, x='DateTime', y='ConfirmedCases', color='continent')
_ = fig2.update_layout(
    yaxis_type="log",
    title_text=f'COVID-19 Cumulative Confirmed Cases by Continent [Updated: {TRAIN_END}]'
)
fig2.show()